# Fixation mapping and processing, exploring the distribution and sequence of visual attention

- mapping fixations to real-world objects and screen contents
- processing the mapped fixations (semantically)
- exploring the distribution and sequence of visual attention using the mapped fixations with metrics and interactive plots

## map fixations to real-world objects using command line tool:

Using file: data/export_fixation.tsv

Example command: `! python fixation_mapping.py C:/Users/admin/detectron2 F:/videos/recording30_full.mp4 F:/exports/export_fixation.tsv F:/exports/data `

Result file: data/mapped_fixation.tsv

## process the mapped fixation file: group the target objects to object groups

The objects were classified into four categories: cell phone, building, surroundings, and others.

Using file: data/mapped_fixation.tsv  
Result file: data/replaced_mapped_fixation.tsv


In [1]:
import pandas as pd
df = pd.read_csv('data/mapped_fixation.tsv', sep='\t')

# categories manually defined, for this case specifically
allowed_list = []
building_class = ['window-blind', 'window', 'ceiling', 'building', 'wall',
                  'house', 'wall-brick', 'wall-stone', 'wall-wood']
surrounding_class = ['tree', 'fence', 'sky', 'pavement', 'grass',
                     'dirt', 'rock', 'road', 'river', 'sand',
                     'person', 'bicycle', 'car', 'motorcycle', 'bus',
                     'train', 'truck', 'traffic light', 'stop sign', 'parking meter']
allowed_list.extend(building_class)
allowed_list.extend(surrounding_class)
allowed_list.append('cell phone')
# replace things, anything out out the "alloed list" is mapped to "others"
df.at[~df["target"].isin(allowed_list), "target"] = "others"
df.replace(building_class, 'building', inplace=True)
df.replace(surrounding_class, 'surroundings', inplace=True)
# write to new file
df.to_csv('data/replaced_mapped_fixation.tsv', sep='\t', index=False)

## add screen contents to the mapped fixations using the command line tool:
Using file: data/replaced_mapped_fixations.tsv  

Example command: `! python screen_recording_match.py "F:/videos/image_pool" "F:/videos/screen.mp4" "F:/exports/mapped/replaced_mapped_fixations.tsv" 201839`

Result file: data/screen_replaced_mapped_fixation.tsv

## process the mapped fixation file: group screen contents, and integrate them with the objects
Using file: data/screen_replaced_mapped_fixation.tsv  
Result file: data/grouped_screen_replaced_mapped_fixation.tsv

In [2]:
df = pd.read_csv('data/screen_replaced_mapped_fixation.tsv', sep='\t')
df.head()

,Recording timestamp,Eye movement type index,Gaze event duration,Fixation point X,Fixation point Y,target,phone_x,phone_y,best_match,screentime
0,119832.0,105,620.0,1007.0,509.0,surroundings,NaN,NaN,NaN,0
1,120222.0,106,400.0,857.0,356.0,cell phone,0.974220,0.210277,map_ar_2.jpg,88
2,120842.0,107,80.0,1678.0,11.0,surroundings,NaN,NaN,NaN,0
3,121262.0,108,440.0,811.0,514.0,cell phone,0.573007,0.161089,map_ar_2.jpg,1128
4,123471.0,109,3799.0,919.0,312.0,cell phone,0.888017,0.129890,map_ar_2.jpg,3337


In [3]:
# make column new_target, replace "cell phone" with the screen content identified in best_match
df['new_target'] = df['target']
df.loc[(df['target'] == 'cell phone'),'new_target'] = df['best_match']
df.new_target.unique()

array(['surroundings', 'map_ar_2.jpg', 'building', 'info_menzis.jpg',
       'view_note.jpg', 'others', 'sat_image.jpg', 'sat_image_2.jpg',
       'take_note.jpg', 'map_ar.jpg', 'old_map.jpg', 'old_map_2.jpg',
       'info_tunnel.jpg', 'photo_itc.jpg', 'info_itc.jpg'], dtype=object)

In [4]:
# process / group / rename the screen contents
import re
df.new_target = df.new_target.apply(lambda x: 'app_' + x.split('.')[0] if x.split('.')[-1]=='jpg' else x)
df.new_target = df.new_target.apply(lambda x: x.split('_', 1)[-1])
df.new_target = df.new_target.apply(
        lambda x: x.split('_')[0] if x.startswith('info') else x)
df.new_target = df.new_target.apply(
        lambda x: x[:-2] if re.search(r'\d+$', x) is not None else x)
df.new_target.replace(['photo_itc', 'sat_image'], 'info', inplace=True)
df.to_csv('data/grouped_screen_replaced_mapped_fixation.tsv', sep='\t', index=False)
df.new_target.unique()

array(['surroundings', 'map_ar', 'building', 'info', 'view_note',
       'others', 'take_note', 'old_map'], dtype=object)

## visualize the distribution and sequence of visual attention:
Distribution metrics: total count, total duration, mean duration  
Sequence metrics: switch frequence; Sequence plot (timeline)  

Using file: data/grouped_screen_replaced_mapped_fixation.tsv

In [5]:
import plotly.graph_objects as go 
df = pd.read_csv('data/grouped_screen_replaced_mapped_fixation.tsv', sep='\t')
df.head()

,Recording timestamp,Eye movement type index,Gaze event duration,Fixation point X,Fixation point Y,target,phone_x,phone_y,best_match,screentime,new_target
0,119832.0,105,620.0,1007.0,509.0,surroundings,NaN,NaN,NaN,0,surroundings
1,120222.0,106,400.0,857.0,356.0,cell phone,0.974220,0.210277,map_ar_2.jpg,88,map_ar
2,120842.0,107,80.0,1678.0,11.0,surroundings,NaN,NaN,NaN,0,surroundings
3,121262.0,108,440.0,811.0,514.0,cell phone,0.573007,0.161089,map_ar_2.jpg,1128,map_ar
4,123471.0,109,3799.0,919.0,312.0,cell phone,0.888017,0.129890,map_ar_2.jpg,3337,map_ar


In [6]:
# plotting colors
color_dict = {'surroundings': 'rgb(204, 235, 197)',
              'others': 'rgb(203, 203, 203)',
              'building': "rgb(128, 177, 211)",
              'cell phone': 'rgb(253, 180, 98)'}
switch_color_dict = {'cell phone - building': 'rgb(253, 180, 98)',
        'cell phone - other objects': 'rgb(204, 235, 197)'}


### pie chart of total fixation duration, and bar chart for mean fixation duration
an example to compare the attention distribution between environment objects and phone. Same comparison can be done for different screen contents


In [7]:
# aggregate count and duration 
target_aggregate = df.groupby('target').agg(
        {'Recording timestamp': 'count', 'Gaze event duration': ['sum', 'mean']}).reset_index()
target_aggregate.columns = target_aggregate.columns.droplevel()
target_aggregate.columns = ['target', 'total count', 'total duration', 'mean duration']

In [8]:
import plotly.graph_objects as go 
import plotly.express as px
import bisect
import numpy as np 

In [9]:
# pie for total duration
fig = go.Figure()

colors = np.array([''] * len(target_aggregate['target']), dtype = object)
for i in np.unique(target_aggregate['target']):
    colors[np.where(target_aggregate['target'] == i)] = color_dict[str(i)]
fig.add_trace(
    go.Pie(
        labels=target_aggregate['target'],
        values=round(target_aggregate['total duration']/1000),
        marker_colors=colors,
        textinfo='value+percent'
        )
    )
fig.update_layout(
    legend=dict(orientation='h'),
    title_text='fixation duration (sec)'
)
fig.show()

In [10]:
# bar for mean duration
fig = px.bar(target_aggregate, x='target', y='mean duration', title='mean fixation duration',
             color_discrete_sequence=["rgb(131, 198, 212)"]
             )
fig.update_layout({'plot_bgcolor': 'rgba(250, 250, 250, 1)'})
fig.show()

### bar chart for switch frequency
switch frequency = # of switches per minute  
switch: a change of fixation target. 

In [11]:
# calculate switch
df['next'] = df['target'].shift(-1)
df['switch'] = ((df['next'] == 'cell phone') & (df['target'].isin(['surroundings']))) | \
                ((df['next'].isin(['surroundings'])) & (df['target'] == 'cell phone'))
df['switch_2'] = ((df['next'] == 'cell phone') & (df['target'].isin(['building']))) | \
                    ((df['next'].isin(['building'])) & (df['target'] == 'cell phone'))
# switch frequency
length = df.at[len(df) - 1, 'Recording timestamp'] - df.at[0, 'Recording timestamp']
m_marks = range(int(df.at[0, 'Recording timestamp']), int(df.at[len(df) - 1, 'Recording timestamp']), 60000)
df['left'] = df.apply(lambda x: bisect.bisect_left(m_marks, x['Recording timestamp']), axis=1)
df.at[0, 'left'] = 1  # set first row
freq_df = df.groupby('left').agg({'Recording timestamp': 'count', 'switch': 'sum', 'switch_2': 'sum'}).reset_index()
freq_df.columns = ['minute', 'fixation count', 'switch count', 'switch 2 count']

In [12]:
# plot them
fig = go.Figure()
fig.add_trace(go.Bar(x=freq_df['minute'], y=freq_df['switch count'],
                        name='cell phone - other objects',
                        marker=dict(color=switch_color_dict['cell phone - other objects']),
                        legendgroup='cell phone - other objects'))
fig.add_trace(go.Bar(x=freq_df['minute'], y=freq_df['switch 2 count'],
                        name='cell phone - building',
                        marker=dict(color=switch_color_dict['cell phone - building']),
                        legendgroup='cell phone - building'))
fig.update_layout(barmode='stack',
                  legend=dict(orientation='h'),
                  yaxis_title="# of switches",
                  xaxis_title='minute',
                  xaxis=dict(tickmode='linear', tick0=1, dtick=1),
                  plot_bgcolor='rgb(250, 250, 250)',
                  title_text='# of switches between cell phone and environment per minute')
                  
fig.show()

### sequence plot to explore the fixation sequence 
plot can be explored interactively (pan zoom etc.)

In [13]:
sequence_color_dict = {'info': 'rgb(253, 180, 98)',
              'map_ar': 'rgb(144, 211, 199)',  # green-ish
              'old_map': "rgb(255, 237, 111)", # yellow-ish
              'take_note': 'rgb(252, 205, 229)',
              'view_note': 'rgb(190, 186, 218)',
              'surroundings': 'rgb(204, 235, 197)',
              'building': 'rgb(128, 177, 211)', 
              'others': 'rgb(203, 203, 203)'}

In [14]:
df['start'] = (df['Recording timestamp'] - df['Gaze event duration']/2) / 1000
df['start'] = df['start'] - df['start'].min()
df['Gaze event duration'] = df['Gaze event duration'] / 1000
fig = go.Figure(
    layout = {
        'barmode': 'stack',
        'xaxis': {'automargin': True, 'title': {'text': 'time into section (sec)'}},
        'yaxis': {'automargin': True,
                  'categoryorder': 'array',
                  'categoryarray': ['others', 'surroundings', 'building',
                                    'view_note', 'take_note',
                                    'map_ar', 'info']},
        'plot_bgcolor': 'rgb(250, 250, 250)'}
)
for target, target_df in df.groupby('new_target'):
    fig.add_bar(x=target_df['Gaze event duration'],
                y=target_df['new_target'],
                base=target_df.start,
                orientation='h',
                showlegend=False,
                marker=dict(color=sequence_color_dict[target]),
                name=target)
fig.show()